<a href="https://colab.research.google.com/github/MariaCamilaPatinoJaramillo/Signal-3/blob/main/Miniproyecto_entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Miniproyecto 1 analisis de señales EMG y extraccion de caracteristicas 

* **Jesus David Restrepo Martinez**

* **Daniel Arturo Vega Hernandez**

* **Maria Camila Patiño Jaramillo**

**Tratamiento de Señales III**

*Universidad de Antioquia*

*Prof. Hernán Felipe García Arias, PhD*

2021-2

# Primera parte para la extraccion de caracteristicas y creacion de la matriz para la señal completa 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

In [3]:
ruta = '/content/drive/MyDrive/Miniproyecto-Señales3/'
data = loadmat(ruta+'S1_20140620T021349.mat')

In [4]:
ruta = '/content/drive/MyDrive/Miniproyecto-Señales3/'
data = loadmat(ruta+'S2_20140623T203911.mat')

In [5]:
ruta = '/content/drive/MyDrive/Miniproyecto-Señales3/'
data = loadmat(ruta+'S3_20140623T192807.mat')

In [6]:
# Llave para acceder al diccionario y obtener los datos correspondientes a EMG
print('Keys: ',data.keys())

data_EMG = data['data_EMG']
fs = 4e3
Ts = 1./fs

print('N (length),\t N_Class\t Trials: ',np.shape(data_EMG) )



Keys:  dict_keys(['__header__', '__version__', '__globals__', 'data_ACC', 'data_EMG'])
N (length),	 N_Class	 Trials:  (20000, 6, 189)


In [7]:
def featureExtractionEMG(sampleSignal):
  # Esta función toma como argumento de entrada una señal EMG de 20000 muestras y retorna 13 caracteristicas relacionadas a la señal
  # a. Removemos el nivel DC
  nivelDC = np.mean(sampleSignal)
  sampleSignal = sampleSignal-nivelDC
  # b. Normalicemos las señales para que tengan amplitud unitaria
  maxSignal = np.abs(np.max(sampleSignal))
  sampleSignal = sampleSignal/maxSignal
  # Realicemos el análisis STFT
  f, t, Zxx = signal.stft(sampleSignal, fs, nperseg=600)

  rms = np.sqrt((np.sum(sampleSignal)**2)/len(sampleSignal))
  mae = np.sum(np.abs(sampleSignal))/len(sampleSignal)
  # En la matriz Zxx se tiene una matriz de #defrecs * #times
# Zxx[i,j], sería el espectro en la frecuencia[i] y el tiempo [j]
  absZxx = np.abs(Zxx)
  Pmax_Zxx = np.max(absZxx,axis=1) # dB

#Para calcular los cruces por cero
  i=0
  cont=0
  for n in range (0,(len(sampleSignal)-1)):
    if( (sampleSignal[n]>0 and sampleSignal[n+1]<0) or (sampleSignal[n]<0 and sampleSignal[n+1]>0)):
      cont=i+1
      i=i+1
    else:
      cont=i+0

  Pmax_Zxx_dB = 20*np.log10(Pmax_Zxx)
  idx = np.argsort(Pmax_Zxx_dB)
  maximos = Pmax_Zxx_dB[idx]
  auxPot = maximos[-5:]
  frecuencias = f[idx]
  fPmax_Zxx = frecuencias[-5:]
  feature_set = np.zeros((13,))
  feature_set[0] = rms
  feature_set[1] = mae
  feature_set[2:7] = auxPot
  feature_set[7:12] = fPmax_Zxx
  feature_set[12]=cont
  return feature_set

In [8]:
# Las dimensiones de data EMG corresponden a 20000,6,189 donde:
#20000 corresponde a los datos
# 6 el tipo de movimiento
# 189 el numero de intentos por ese movimiento 

#lo que se esta haciendo es crear la matriz de caracteristicas para toda la señal x y tambien se crea un vector columna que representa el tipo de movimiento T
L_Signal, Classes, Ntrials = np.shape(data_EMG)
D = 13 # número de características

X = np.zeros((Classes*Ntrials,D))
t = np.zeros((Classes*Ntrials,1))
pos = 0
for clase in range(0,Classes):
  for n in range(0,Ntrials):
    sampleSignal = data_EMG[:,clase,n]
    # Luego le extraemos las D características a cada señal del experimento
    x_n = featureExtractionEMG(sampleSignal)
    X[pos, :] = x_n
    t[pos] = clase
    pos = pos + 1

In [9]:
#Aqui ya tenemos la señal original con todas las caracteristicas que serian 13 

print(np.shape(X))

(1134, 13)


# Segunda parte usando las funciones de wavelet, extraccion de caracteristicas y formacion matriz gigante

In [10]:
import pywt  #nombre de la libreria 
import numpy as np
data =sampleSignal
np.shape(data)

(20000,)

Para la parte de las funciones wavelet se va a crear una matriz que va a tener 
+ N filas : que corresponden a los niveles de descomposicion 
+ 2 columnas: donde una van a ser los coeficientes de aproximacion(Pasaaltas) y la otra los coeficientes de detalle (Pasabajas)  

In [13]:
import scipy 
from scipy.stats import entropy
!pip install antropy
import antropy as ant

def calculate_entropy(list_values):
  entropy_val =ant.perm_entropy(signal, normalize=True)
  return entropy_val

def calculate_statistics(list_values):
  n5 = np.nanpercentile(list_values, 5)
  n25 = np.nanpercentile(list_values, 25)
  n75 = np.nanpercentile(list_values, 75)
  n95 = np.nanpercentile(list_values, 95)
  median = np.nanpercentile(list_values, 50)
  mean = np.nanmean(list_values)
  std = np.nanstd(list_values)
  var = np.nanvar(list_values)
  rms = np.nanmean(np.sqrt(list_values**2))
  return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
  zero_crossing_indices = np.where(np.diff(np.signbit(list_values)))[0]
  no_zero_crossings = len(zero_crossing_indices)
  mean_crossing_indices = np.where(np.diff(np.signbit(list_values-np.nanmean(list_values))))[0]
  no_mean_crossings = len(mean_crossing_indices)
  return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):
  entropy = calculate_entropy(list_values)
  crossings = calculate_crossings(list_values)
  statistics = calculate_statistics(list_values)
  return [entropy] + crossings + statistics

  

# PARA LA DESCOMPOSICION #1 

In [14]:
import pywt
import numpy as np
import sys


X_1 = np.zeros((Classes*Ntrials,12))
X_1_1 = np.zeros((Classes*Ntrials,12))
t_1 = np.zeros((Classes*Ntrials,1))
pos=0
bu=[]
for clase in range(0,Classes):
  for n in range(0,Ntrials):
    signal = data_EMG[:,clase,n]
     
    nDesc =1
    waveletname = 'db'+str(nDesc)

    for i in range(nDesc):
      (data, coeff_d) = pywt.dwt(signal, waveletname)
      conjunto1Features1 = get_features(data)
      conjunto1Features1_1 = get_features(coeff_d)
      X_1[pos, :] = conjunto1Features1
      X_1_1[pos,:]=conjunto1Features1_1
      t_1[pos] = clase
      pos = pos + 1

data_des1=data

  


In [15]:

np.set_printoptions(threshold=sys.maxsize)
print(np.shape(X_1))
#print(X_1)



(1134, 12)


# Para la descomposicion #2

In [16]:
import pywt
import numpy as np
import sys

señal2=np.zeros((len(data_des1),Classes,Ntrials))
for z in range(0,len(data_des1)):
  for clase in range(0,Classes):
    for n in range(0,Ntrials):
    
      señal2[z,clase,n]=data_des1[z]

X_2 = np.zeros((Classes*Ntrials,12))
t_2 = np.zeros((Classes*Ntrials,1))
pos=0
data_des2=[]
for clase in range(0,Classes):
  for n in range(0,Ntrials):
    signal1 = señal2[:,clase,n]
     
    nDesc =1
    waveletname = 'db'+str(nDesc)

    for i in range(nDesc):
      (data1, coeff_d) = pywt.dwt(signal1, waveletname)
      conjunto1Features2 = get_features(data1)
      X_2[pos, :] = conjunto1Features2
      t_2[pos] = clase
      pos = pos + 1
    
  data_des2=data1

#Descomposicion #3

In [17]:
import pywt
import numpy as np
import sys

señal3=np.zeros((len(data_des2),Classes,Ntrials))
for z in range(0,len(data_des2)):
  for clase in range(0,Classes):
    for n in range(0,Ntrials):
    
      señal3[z,clase,n]=data_des2[z]

X_3 = np.zeros((Classes*Ntrials,12))
t_3 = np.zeros((Classes*Ntrials,1))
pos=0
data_des3=[]
for clase in range(0,Classes):
  for n in range(0,Ntrials):
    signal2 = señal3[:,clase,n]
     
    nDesc =1
    waveletname = 'db'+str(nDesc)

    for i in range(nDesc):
      (data2, coeff_d) = pywt.dwt(signal2, waveletname)
      conjunto1Features3 = get_features(data)
      X_3[pos, :] = conjunto1Features3
      t_3[pos] = clase
      pos = pos + 1
    
  data_des3=data2


# Ya se tienen la matriz de caracteristicas para las 3 descomposiciones de la funciones wavelet

Impresion de los tamaños de cada matriz de caracteristicas


In [18]:
print(np.shape(X))  #vector de caracteristicas de la señal original
print(np.shape(X_1_1)) # vector de caracteristicas para la primera descomposicion de los coef de pasa altas
print(np.shape(X_1)) # vector de caracteristicas para la primera descomposicion
print(np.shape(X_2))  # vector de caracteristicas para la segunda descomposicion
print(np.shape(X_3))  # vector de caracteristicas para la tercera descomposicion

(1134, 13)
(1134, 12)
(1134, 12)
(1134, 12)
(1134, 12)


In [19]:
# Importa pandas 
import pandas as pd
matriz_completa=np.concatenate((X,X_1,X_2,X_3,X_1_1,t_1), axis=1)
print(np.shape(matriz_completa))



(1134, 62)


#Generacion del super vector con todas las caracteristicas de las señanes EMG 



In [20]:
# Importa pandas 
import pandas as pd
Nombres = ['RMS','MAE','P1','P2','P3','P4','P5','F1','F2','F3','F4','F5','Num Cruces Por Cero','entropia', 'cruces por cero', 'cruce con la media', 'percentil 5','percentil 25','percentil75','percentil95','mediana','media','std', 'varianza', 'rms','entropia', 'cruces por cero', 'cruce con la media', 'percentil 5','percentil 25','percentil75','percentil95','mediana','media','std', 'varianza', 'rms','entropia', 'cruces por cero', 'cruce con la media', 'percentil 5','percentil 25','percentil75','percentil95','mediana','media','std', 'varianza', 'rms','entropia', 'cruces por cero', 'cruce con la media', 'percentil 5','percentil 25','percentil75','percentil95','mediana','media','std', 'varianza', 'rms','Tipo de Mov']

dataFrame = pd.DataFrame(data = matriz_completa,columns= Nombres)


# Creacion de dataframe 


In [21]:
dataFrame

,RMS,MAE,P1,P2,P3,P4,P5,F1,F2,F3,F4,F5,Num Cruces Por Cero,entropia,cruces por cero,cruce con la media,percentil 5,percentil 25,percentil75,percentil95,mediana,media,std,varianza,rms,entropia,cruces por cero,cruce con la media,percentil 5,percentil 25,percentil75,percentil95,mediana,media,std,varianza,rms,entropia,cruces por cero,cruce con la media,percentil 5,percentil 25,percentil75,percentil95,mediana,media,std,varianza,rms,entropia,cruces por cero,cruce con la media,percentil 5,percentil 25,percentil75,percentil95,mediana,media,std,varianza,rms,Tipo de Mov
0,2.311176e-15,0.294098,-25.461826,-22.838421,-21.593740,-13.185073,-13.159219,106.666667,40.000000,60.000000,53.333333,46.666667,535.0,0.616959,565.0,533.0,-0.022703,-0.004099,0.061013,0.080780,0.028690,0.028736,0.035547,0.001264,0.037181,0.563345,732.0,728.0,-0.27186,-0.100881,0.11983,0.278846,0.010564,0.006975,0.190191,0.036173,0.137862,0.563345,756.0,754.0,-0.194798,-0.072235,0.085663,0.196819,0.00776,0.004932,0.136176,0.018544,0.098478,0.616959,1660.0,1649.0,-0.003023,-0.001395,0.001395,0.003023,0.000000,1.209234e-06,0.001860,0.000003,0.001506,0.0
1,7.492481e-15,0.316546,-23.777920,-21.882633,-20.762850,-12.560300,-12.426907,100.000000,40.000000,60.000000,46.666667,53.333333,540.0,0.608746,538.0,538.0,-0.022470,-0.003402,0.060083,0.079849,0.027992,0.028447,0.034821,0.001213,0.036600,0.563345,732.0,728.0,-0.27186,-0.100881,0.11983,0.278846,0.010564,0.006975,0.190191,0.036173,0.137862,0.563345,756.0,754.0,-0.194798,-0.072235,0.085663,0.196819,0.00776,0.004932,0.136176,0.018544,0.098478,0.608746,1514.0,1520.0,-0.002791,-0.001395,0.001395,0.002791,0.000000,2.116159e-06,0.001813,0.000003,0.001473,0.0
2,1.218392e-15,0.263807,-24.561016,-23.143446,-21.286972,-13.982896,-13.716889,80.000000,40.000000,60.000000,53.333333,46.666667,576.0,0.617770,554.0,566.0,-0.023866,-0.004099,0.058920,0.081942,0.027527,0.028184,0.035760,0.001279,0.036930,0.563345,732.0,728.0,-0.27186,-0.100881,0.11983,0.278846,0.010564,0.006975,0.190191,0.036173,0.137862,0.563345,756.0,754.0,-0.194798,-0.072235,0.085663,0.196819,0.00776,0.004932,0.136176,0.018544,0.098478,0.617770,1670.0,1664.0,-0.003023,-0.001395,0.001395,0.003023,0.000000,4.488117e-06,0.001961,0.000004,0.001565,0.0
3,2.700559e-15,0.236866,-24.106740,-22.832281,-22.349908,-14.643568,-14.472559,66.666667,60.000000,40.000000,46.666667,53.333333,608.0,0.642996,632.0,594.0,-0.024796,-0.002239,0.057293,0.081245,0.027294,0.027793,0.035071,0.001230,0.036288,0.563345,732.0,728.0,-0.27186,-0.100881,0.11983,0.278846,0.010564,0.006975,0.190191,0.036173,0.137862,0.563345,756.0,754.0,-0.194798,-0.072235,0.085663,0.196819,0.00776,0.004932,0.136176,0.018544,0.098478,0.642996,2013.0,2013.0,-0.003488,-0.001395,0.001395,0.003488,0.000000,-4.557880e-06,0.002180,0.000005,0.001716,0.0
4,1.984597e-15,0.280108,-23.311912,-21.536340,-20.635537,-13.342430,-13.037959,66.666667,60.000000,40.000000,53.333333,46.666667,642.0,0.638023,630.0,620.0,-0.027121,-0.003634,0.057293,0.084035,0.027527,0.027512,0.036399,0.001325,0.037011,0.563345,732.0,728.0,-0.27186,-0.100881,0.11983,0.278846,0.010564,0.006975,0.190191,0.036173,0.137862,0.563345,756.0,754.0,-0.194798,-0.072235,0.085663,0.196819,0.00776,0.004932,0.136176,0.018544,0.098478,0.638023,1898.0,1898.0,-0.003721,-0.001395,0.001395,0.003721,0.000000,-5.813623e-07,0.002236,0.000005,0.001763,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,4.616072e-16,0.176549,-22.046743,-21.697849,-21.176839,-17.936285,-17.292164,120.000000,86.666667,93.333333,53.333333,46.666667,679.0,0.555216,661.0,675.0,-0.138975,-0.066886,0.079617,0.148927,0.008458,0.005736,0.096820,0.009374,0.079132,0.563345,732.0,728.0,-0.27186,-0.100881,0.11983,0.278846,0.010564,0.006975,0.190191,0.036173,0.137862,0.563345,756.0,754.0,-0.194798,-0.072235,0